In [2]:
import os
import pandas as pd
from functions import get_all_origins, find_pattern_for_quantity, convert_to_grams, relation_qnt_preco, remove_spaces, clean_text
import re
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = get_all_origins()

In [4]:
def filter_title(df, keyword):
    filtered_df = df[df['titulo'].str.contains(keyword, case=False)]
    return filtered_df

def exclude_in_title(df, keyword):
    filtered_df = df[~df['titulo'].str.contains(keyword, case=False)]
    return filtered_df

filter_title(df, 'pretreino')

,ref,titulo,preco,link_imagem,link_produto,ing_date,nome,marca,preco_numeric,quantidade,preco_qnt,especificacao,especificacao_rota
71,5646f02c,full tilt pretreino 150g,"69,99",https://www.iridiumlabs.com.br/cdn/shop/produc...,https://www.iridiumlabs.com.br/products/full-t...,18/11/2023,full tilt pré-treino - 150g,iridium labs,69.99,150.0,0.467,NaN,beta alanina cafein vit
72,249fb6b0,minotauro ultimate pretreino 300g,"99,99",https://www.iridiumlabs.com.br/cdn/shop/produc...,https://www.iridiumlabs.com.br/products/minota...,18/11/2023,minotauro ultimate pré-treino - 300g,iridium labs,99.99,300.0,0.333,NaN,beta alanina arginin pretreino
73,695d9587,minotauro pretreino 300g,"69,99",https://www.iridiumlabs.com.br/cdn/shop/produc...,https://www.iridiumlabs.com.br/products/minota...,18/11/2023,minotauro pré-treino - 300g,iridium labs,69.99,300.0,0.233,NaN,vit
74,32252fc2,minotauro extreme pretreino 300g,"89,99",https://www.iridiumlabs.com.br/cdn/shop/produc...,https://www.iridiumlabs.com.br/products/minota...,18/11/2023,minotauro extreme pré-treino - 300g,iridium labs,89.99,300.0,0.300,NaN,arginin
347,5c4336e3,panic pretreino nova formula 150g,"65,55",https://adaptogen.com.br/wp-content/uploads/20...,https://adaptogen.com.br/panic-pre-treino-nova...,17/11/2023,panic pré-treino – nova formula 150g,adaptogen,65.55,150.0,0.437,NaN,pretreino
348,bf5e6b83,panic pretreino nova formula 300g,"151,05",https://adaptogen.com.br/wp-content/uploads/20...,https://adaptogen.com.br/panic-pre-treino-nova...,17/11/2023,panic pré-treino – nova formula 300g,adaptogen,151.05,300.0,0.504,NaN,pretreino
350,020de9b9,magic pretreino comam frutas 183g,"137,75",https://adaptogen.com.br/wp-content/uploads/20...,https://adaptogen.com.br/magic-pre-treino-coma...,17/11/2023,magic pré-treino comam frutas 183g,adaptogen,137.75,183.0,0.753,NaN,pretreino
351,04200654,panic pretreino pack com 20 saches de 10g,"132,05",https://adaptogen.com.br/wp-content/uploads/20...,https://adaptogen.com.br/panic-pre-treino-pack...,17/11/2023,panic pré-treino pack com 20 sachês de 10g,adaptogen,132.05,10.0,13.205,NaN,pretreino
419,5b8b7dfa,combo w100 creatina pretreino,"396,10",https://nutrata.com.br/wp-content/uploads/2023...,https://nutrata.com.br/produto/combo-w100-crea...,18/11/2023,combo w100 + creatina + pré-treino,nutrata,396.10,NaN,NaN,NaN,whey concentrado creatina
631,bc10d799,pretreino haze hardcore 300g growth supplements,"103,50",https://www.gsuplementos.com.br/upload/produto...,https://www.gsuplementos.com.br/pre-treino-haz...,13/11/2023,pré-treino haze hardcore 300g,growth supplements,103.50,300.0,0.345,NaN,pretreino
